In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import json
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from dotenv import load_dotenv


In [ ]:
try:

    llm  = ChatOllama(model="gemma3:27b", temperature=0, base_url="http://192.168.133.192:11435")
except Exception as e:

    print(f"Erro ao inicializar o Ollama. Certifique-se de que o Ollama está em execução e o modelo 'gemma3' está disponível.")

    print(f"Detalhes do erro: {e}")

    exit()

ResponseError: <!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html><head>
<meta type="copyright" content="Copyright (C) 1996-2023 The Squid Software Foundation and contributors">
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<title>ERROR: Cache Access Denied</title>
<style type="text/css"><!-- 
 /*
 * Copyright (C) 1996-2023 The Squid Software Foundation and contributors
 *
 * Squid software is distributed under GPLv2+ license and includes
 * contributions from numerous individuals and organizations.
 * Please see the COPYING and CONTRIBUTORS files for details.
 */

/*
 Stylesheet for Squid Error pages
 Adapted from design by Free CSS Templates
 http://www.freecsstemplates.org
 Released for free under a Creative Commons Attribution 2.5 License
*/

/* Page basics */
* {
	font-family: verdana, sans-serif;
}

html body {
	margin: 0;
	padding: 0;
	background: #efefef;
	font-size: 12px;
	color: #1e1e1e;
}

/* Page displayed 

In [ ]:


system_prompt_text = """

Você é um especialista em Reconhecimento de Entidades Nomeadas (NER) treinado para identificar e classificar entidades em textos jurídicos brasileiros.
Sua tarefa é analisar o texto fornecido e retornar cada palavra (token) acompanhada de uma etiqueta NER no formato BIO.
As etiquetas possíveis são:

- O: Para palavras que não pertencem a nenhuma entidade.
- B-Organizacao: Para a primeira palavra em uma frase de uma entidade do tipo "Organização" (ex: Ministério Público, Tribunal de Justiça).
- I-Organizacao: Para palavras subsequentes dentro de uma frase para entidade do tipo "Organização".
- B-Jurisprudencia: Para a primeira palavra em uma frase de uma entidade do tipo "Jurisprudência" (ex: Súmula 385 STJ, Recurso Especial nº 1.234.567).
- I-Jurisprudencia: Para palavras subsequentes dentro de uma frse para entidade do tipo "Jurisprudência".
- B-Pessoa: Para a primeira palavra em uma frase de uma entidade do tipo "Pessoa" (ex: João da Silva, Maria Oliveira).
- I-Pessoa: Para palavras subsequentes dentro de uma frase para entidade do tipo "Pessoa".
- B-Tempo: Para a primeira palavra em uma frase de uma entidade do tipo "Tempo" (ex: 01 de janeiro de 2023, segunda-feira, prazo de 15 dias).
- I-Tempo: Para palavras subsequentes dentro de uma frase para entidade do tipo "Tempo".
- B-Local: Para a primeira palavra em uma frase de uma entidade do tipo "Local" (ex: São Paulo, Comarca de Campinas, Brasil).
- I-Local: Para palavras subsequentes dentro de uma frase para entidade do tipo "Local".
- B-Legislacao: Para a primeira palavra em uma frase de uma entidade do tipo "Legislação" (ex: Código Civil, Lei nº 8.666/93, Art. 5º da Constituição Federal).
- I-Legislacao: Para palavras subsequentes dentro de uma frase para entidade do tipo "Legislação".

A entrada será uma lista de listas de string no seguinte formato, exceto pelo termo 'FraseN' no início de cada lista de frases:
[
    Frase1: ['O', 'Dia', 'Está', 'Lindo', 'Hoje'],
    Frase2: ['O', 'Relator', 'Está', 'Ciente', 'Do', 'Contrato'],
    Frase3: ['No', 'caso', 'em', 'debate', 'nestes', 'autos', ',', 'a', 'guarda']
    Frase4: ['Eu', 'sou', 'uma', 'pessoa', 'muito', 'inteligente', '-', 'disse', 'o', 'homem']
    Frase5: ['O', 'arts','.','58','e','67', ',', 'Lei', '8666/94', ',' , '186', 'e', '927', 'do', 'Código', 'Civil', ')', '.' ]
    Frase6: ['Encontra-se', 'em', 'consonância', 'com', 'o', 'fundamento', 'acolhido', 'pelo', 'STF']
    Frase7: ['Confiram-se', 'as', 'seguintes', 'decisões', ':', 'Rcl', '13941', 'MC', '/', ',' , 'Relator', 'Ministros', 'Cezar', 'Peluso', 'DJE', '31/08/2012']
]

A saída DEVE ser estritamente no formato, e apenas nesse formato e nada mais:
{{

    "Frase1": ["O", "O", "O", "O", "B-TEMPO"]
    "Frase2": ["O", "B-PESSOA", "O", "O", "O"]
    "Frase3": ["O","O","O","O","O","O","O","O","O"]
    "Frase4": ["B-PESSOA", "O", "O", "I-PESSOA", "O", "O", "O", "O", "O", "I-PESSOA"]
    "Frase5": ["O", "B-LEGISLACAO", "I-LEGISLACAO", "I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","O","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","I-LEGISLACAO","O","O"]
    "Frase6": ["O", "O", "O", "O", "O", "O", "O", "O", "B-ORGANIZAÇÃO"]
    "Frase7": ["O", "O", "O", "O", "O", "B-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "I-JURISPRUDENCIA", "O", "O", "O", "B-PESSOA", "I-PESSOA", "O", "B-TEMPO"]
}}

Na qual cada posicao do vetor representa a respectiva entidade nomeada da frase. 

NÃO RESPONDA EM MARKDOWN, RESPONDA EM STRING SIMPLES
"""

user_prompt_text = """Analise o seguinte texto e forneça a classificação NER para cada palavra no formato especificado:


TEXTO DE ENTRADA:
{texto_entrada}"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    ("user", user_prompt_text)
])

chain = prompt_template | llm | StrOutputParser()
texto_exemplo = """
[
    Frase1: ['Número', 'do', 'Acórdão', 'ACÓRDÃO', '1160/2016', '-', 'PLENÁRIO', 'Relator', 'AUGUSTO', 'NARDES', 'Processo', '006.010/2000-4', 'Tipo', 'de', 'processo', 'TOMADA', 'DE', 'CONTAS', 'SIMPLIFICADA', '(', 'TCSP', ')', 'Data', 'da', 'sessão', '11/05/2016', 'Número', 'da', 'ata', '16/2016', 'Relator', 'da', 'deliberação', 'recorrida', 'Ministra', 'Ana', 'Arraes', '.']
]
"""



In [ ]:
response = chain.invoke({"texto_entrada" : texto_exemplo})
print(response)


KeyboardInterrupt: 